In [3]:
import pandas as pd

In [4]:
books = pd.read_csv("goodbooks-10k/books.csv")
#book_tags30 = pd.read_csv("top30tags/book_tags_01_top30tags.csv")
book_tags10 = pd.read_csv("toptags10/book_data.csv")
tags = pd.read_csv("goodbooks-10k/tags.csv")

In [30]:
def pivot_wide_book_tags(book_tags: pd.DataFrame, tags: pd.DataFrame) -> pd.DataFrame:
    return (
        book_tags
        .merge(tags, on="tag_id")
        .pivot_table(columns="tag_name", index="goodreads_book_id", values="count", fill_value=0)
        .assign(total=lambda d: d.sum(1))
        .apply(lambda x: x/x.total, axis=1)
        .drop(columns=["total"])
    )

def pivot_wide_book_tags_binary(book_tags: pd.DataFrame, tags: pd.DataFrame, cutoff = .1) -> pd.DataFrame:
    df = 1*(pivot_wide_book_tags(book_tags, tags) > cutoff)
    return df

In [31]:
df_wide10 = pivot_wide_book_tags(book_tags10, tags)
df_wide30 = pivot_wide_book_tags(book_tags30, tags)
df_wide10_filter = pivot_wide_book_tags(book_tags10_filter, tags)
df_wide10_filter_binary = pivot_wide_book_tags_binary(book_tags10_filter, tags)

In [32]:
df_wide10_filter_binary

tag_name,childrens,classics,fantasy,historical-fiction,mystery,non-fiction,paranormal,romance,science-fiction,young-adult
goodreads_book_id,,,,,,,,,,
1,0,0,1,0,0,0,0,0,0,1
2,0,0,1,0,0,0,1,0,0,1
3,0,0,1,0,0,0,0,0,0,1
5,0,0,1,0,0,0,0,0,0,1
6,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
31538647,0,0,1,0,0,0,0,0,0,1
31845516,0,0,0,0,0,1,0,0,0,0
32075671,0,0,0,0,0,0,0,0,0,1


In [34]:
1*(df_wide10_filter > .1)

tag_name,childrens,classics,fantasy,historical-fiction,mystery,non-fiction,paranormal,romance,science-fiction,young-adult
goodreads_book_id,,,,,,,,,,
1,0,0,1,0,0,0,0,0,0,1
2,0,0,1,0,0,0,1,0,0,1
3,0,0,1,0,0,0,0,0,0,1
5,0,0,1,0,0,0,0,0,0,1
6,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
31538647,0,0,1,0,0,0,0,0,0,1
31845516,0,0,0,0,0,1,0,0,0,0
32075671,0,0,0,0,0,0,0,0,0,1


## Combine with book
tag proportion and book ids in wide format

In [35]:
def merge_df_wide_with_books(books, df_wide):
    return (
        books
        .filter(items=["book_id", "goodreads_book_id"])
        .merge(df_wide, on = "goodreads_book_id")
    )

def create_book_data_csv(output_name, book_tags_name, books, tags):
    book_tags = pd.read_csv(book_tags_name)
    df_wide = pivot_wide_book_tags(book_tags, tags)
    df = merge_df_wide_with_books(books, df_wide)
    df.to_csv(output_name, index=False)

## Save to csv

In [21]:
create_book_data_csv(output_name = "top10tags/book_data.csv", 
                     book_tags_name = "top10tags/book_tags.csv",
                     books = books,
                     tags = tags)

In [22]:
create_book_data_csv(output_name = "top30tags/book_data.csv", 
                     book_tags_name = "top30tags/book_tags_01_top30tags.csv",
                     books = books,
                     tags = tags)

In [23]:
create_book_data_csv(output_name = "top10tags/book_data_filter.csv", 
                     book_tags_name = "top10tags/book_tags_filter.csv",
                     books = books,
                     tags = tags)

In [36]:
df_binary = merge_df_wide_with_books(books, df_wide10_filter_binary)
df_binary.to_csv("top10tags/book_data_filter_binary.csv", index=False)